# **Pipeline for variant caling in *Mtb***

This Jupyter Notebook has been developed by Rita Nóbrega-Martins (PG46733) for the curicular unit 'Project' of the Master's in Bioinformatics, University of Minho. 
This work has been supervised by Dr. Nuno Osório (ICVS, University of Minho) and Dr. Tiago Beites (i3S, University of Porto).


This pipeline runs on Linux system or WSL-Ubuntu that runs a conda environment named 'MTB_calling'.
This conda environment contains the following tools:
* `bcftools`              version   1.11               
* `bedtools`              version   2.30.0 
* `bwa`                   version   0.7.17
* `bzip2`                 version   1.0.8 
* `fastqc`                version   0.12.1 
* `htslib`                version   1.19.1
* `ipykernel`             version   6.29.3           
* `ipython`               version   8.22.2
* `lofreq`                version   2.1.5
* `mosdepth`              version   0.3.3 
* `multiqc`               version   1.21 
* `parallel`              version   20160622
* `pathogen-profiler`     version   2.0.1 
* `samtools`              version   1.12
* `sra-tools`             version   3.1.0
* `tabixpp`               version   1.1.2               
* `tb-profiler`           version   4.3.0 
* `trimmomatic`           version   0.39
* `vcflib`                version   1.0.9


In [2]:
conda activate MTB_calling

# **1.FASTQ extraction**

**Input:** Text file with a list of SRA accession numbers of the *Mtb* isolates

The funcion `extractSRAfiles()` iterates each line of the text file cointaining the SRA number of a given isolate, creating a directory for each isolate/sample, where information and files regarding that specific isolate will be stored.

Through the tools `prefetch` and `fasterq-dump`, SRA data will be downnloaded from the NCBI database, and converted to FastQ files [1].

**Output:** 2 FastQ files per sample


In [5]:
extractSRAfiles () {
    echo "Extracting FastQ files"
    local sample="$1" 
    local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads"

    while IFS= read -r sample; do
        echo -e "\nSample: $sample"
        mkdir -p "$sra_diretoria/$sample"  
        cd "$sra_diretoria/$sample" || exit 1 

        prefetch "$sample" 
        fasterq-dump "$sample"
        rm $HOME/Projeto_Mtb/SRA_files/sra/"$sample".sra
    
    done < "$sample"
}

In [6]:
extractSRAfiles ~/Projeto_Mtb/list_isolates_SRA.txt

Extracting FastQ files

Sample: DRR034421

2024-06-14T14:02:52 prefetch.3.1.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2024-06-14T14:02:52 prefetch.3.1.0: 1) 'DRR034421' is found locally
2024-06-14T14:02:52 prefetch.3.1.0: 'DRR034421' has 0 unresolved dependencies
spots read      : 606,259
reads read      : 1,212,518
reads written   : 1,212,518

Sample: DRR130077

2024-06-14T14:03:10 prefetch.3.1.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2024-06-14T14:03:10 prefetch.3.1.0: 1) 'DRR130077' is found locally
2024-06-14T14:03:11 prefetch.3.1.0: 'DRR130077' has 0 unresolved dependencies
spots read      : 573,970
reads read      : 1,147,940
reads written   : 1,147,940

Sample: DRR130078

2024-06-14T14:03:31 prefetch.3.1.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2024-06-14T14:03:32 prefetch.3.1.0: 1) 'DRR130078' is found lo

# **2. Reads Pre-processing**

The pre-processing of the reads present in the FASTQ-files extracted consists on extraction of the FASTQC file, which contain a Quality Report, and the trimming of the reads according to its quality. 

The function `quality_control` receives the text file with the list of isolates and iterates through each line/isolate to access the sample folder and through `fastqc` download the FASTQC-file (in a html format) per FASTQ-file, with relevant information for the assessment of the quality of the reads.

The function `trimming_bbduk` iterates through the lines of the text file given as an input to access the 2 FASTQ-files of each isolate/sample and perform a trimming of the reads through the `BBDuk` tool [2] to remove adapters and low-quality reads, as well as reads that may be too short for further analysis.
`trimming_bbduk` returns the 2 FASTQ-files with the trimmed reads per sample.

In [81]:
quality_control () {
  echo 'Quality control'
  local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads"
  local sample_file="$1"
  sed -i 's/\r$//' "$sample_file"  

  if [ ! -f "$sample_file" ]; then
      echo "Sample '$sample_file' file not found."
      return 1
  fi

  parallel -j "$(nproc)" "
      sample={};
      if [ -f \"$sra_diretoria/\$sample/\${sample}_1.fastq\" ] && [ -f \"$sra_diretoria/\$sample/\${sample}_2.fastq\" ]; then
          fastqc \"$sra_diretoria/\$sample/\${sample}_1.fastq\" \"$sra_diretoria/\$sample/\${sample}_2.fastq\";
      else
          echo \"FASTQ not found for: '\$sample'.\";
      fi
    " < "$sample_file"
}

    # vizualização da qualidade - basta o report html ou uso o Qualitymap ou o fastxtoolkit (permitem ver na command line)

In [82]:
quality_control ~/Projeto_Mtb/list_isolates_SRA.txt

Quality control
Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

To silence the citation notice: run 'parallel --citation'.

null
null
Analysis complete for DRR034421_1.fastq
Analysis complete for DRR034421_2.fastq
Started analysis of DRR034421_1.fastq
Approx 5% complete for DRR034421_1.fastq
Approx 10% complete for DRR034421_1.fastq
Approx 15% complete for DRR034421_1.fastq
Approx 20% complete for DRR034421_1.fastq
Approx 25% complete for DRR034421_1.fastq
Approx 30% complete for DRR034421_1.fastq
Approx 35% complete for DRR034421_1.fastq
Approx 40% complete for DRR034421_1.fastq
Approx 45% complete for DRR034421_1.

In [83]:
trimming_bbduk () {
  echo 'Read trimming with BBDuk'
  local sample_file="$1"
  sed -i 's/\r$//' "$sample_file"  

  if [ ! -f "$sample_file" ]; then
      echo "Sample '$sample_file' file not found."
      return 1
  fi

  cat "$sample_file" | xargs -I {} -P "$(nproc)" bash -c '
    sample={};
    echo "Sample: $sample";
    if [ -d "$HOME/Projeto_Mtb/Sample_reads/$sample" ]; then
      if [ -f "$HOME/Projeto_Mtb/Sample_reads/$sample/${sample}_1.fastq" ] && [ -f "$HOME/Projeto_Mtb/Sample_reads/$sample/${sample}_2.fastq" ]; then
        bbduk.sh \
          in1="$HOME/Projeto_Mtb/Sample_reads/$sample/${sample}_1.fastq" in2="$HOME/Projeto_Mtb/Sample_reads/$sample/${sample}_2.fastq" \
          out="$HOME/Projeto_Mtb/Sample_reads/$sample/${sample}_R1_bbduk.fastq" out2="$HOME/Projeto_Mtb/Sample_reads/$sample/${sample}_R2_bbduk.fastq" \
          overwrite=t \
          ref=~/Projeto_Mtb/NGS_helper_files/adapters_combined_256_unique.fasta \
          ftm=5 \
          ktrim=r k=19 mink=8 editdistance=1 editdistance2=1 \
          trimpairsevenly=f removeifeitherbad=t \
          qtrim=r trimq=20 \
          trimpolygright=10 \
          minavgquality=20 \
          minlength=20 ottm=t \
          rename=t ziplevel=1 showspeed=t ;
        rm "$HOME/Projeto_Mtb/Sample_reads/$sample/${sample}"_[1-2].fastq ; 
        echo "Trimming for $sample completed"
      else
        echo "FASTQ not found for: $sample.";
      fi
    else
      echo "Directory for $sample not found.";
    fi
  '
}

In [84]:
trimming_bbduk ~/Projeto_Mtb/list_isolates_SRA.txt

Read trimming with BBDuk
Sample: DRR130077
Sample: DRR130078
Sample: DRR034421
Sample: DRR130093
/usr/share/bbmap/calcmem.sh: line 207: dpkg-architecture: command not found
/usr/share/bbmap/calcmem.sh: line 207: dpkg-architecture: command not found
/usr/share/bbmap/calcmem.sh: line 207: dpkg-architecture: command not found
/usr/share/bbmap/calcmem.sh: line 207: dpkg-architecture: command not found
java -ea -Xmx283m -Xms283m -cp /usr/share/java/bbmap.jar jgi.BBDuk in1=/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_1.fastq in2=/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_2.fastq out=/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_R1_bbduk.fastq out2=/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_R2_bbduk.fastq overwrite=t ref=/home/ritanobrega00/Projeto_Mtb/NGS_helper_files/adapters_combined_256_unique.fasta ftm=5 ktrim=r k=19 mink=8 editdistance=1 editdistance2=1 trimpairsevenly=f removeifeitherbad=t qtrim=r trimq=

# **3. Mapping**

After the pre-processement to ensure we will only work with high-quality reads, the trimmed reads will be mapped against the *Mtb*'s ancestral reference genome.

The function `map_bwa` uses the `bwa mem` [3] and `samtools` to perform the mapping and consequent creation of the BAM file per sample, which will be indexed, sorted by read name, and compressed, to facilitate efficient access and retrieval of read alignments.

In [85]:
map_bwa() {
  echo "Mapping genomes to MTB_anc with bwa mem + sorting BAM by read name"
  
  local sample_file="$1"
  local genome_reference="$HOME/Projeto_Mtb/NGS_helper_files/MTB_anc.fasta"
  local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads"

  if [ ! -f "$sample_file" ]; then
      echo "Sample '$sample_file' file not found."
      return 1
  fi

  while IFS= read -r sample; do
    echo -e "\nMapping Sample: $sample"
    
    if [ ! -f "$sra_diretoria/$sample/${sample}_R1_bbduk.fastq" ] || [ ! -f "$sra_diretoria/$sample/${sample}_R2_bbduk.fastq" ]; then
      echo "Input for '$sample_file' not found."
      continue
    fi

    bwa_threads=$(nproc)
    bwa mem -t "$bwa_threads" "$genome_reference" "$sra_diretoria/$sample/${sample}_R1_bbduk.fastq" "$sra_diretoria/$sample/${sample}_R2_bbduk.fastq" \
    | samtools sort -n -l 1 -@ 1 -o "$sra_diretoria/$sample/$sample.bam"

    samtools view -@ 1 -b "$sra_diretoria/$sample/$sample.bam" | samtools sort -@ 1 -o "$sra_diretoria/$sample/${sample}_sorted.bam"
    samtools index -@ 1 "$sra_diretoria/$sample/${sample}_sorted.bam"
    
    rm "$sra_diretoria/$sample/${sample}_R1_bbduk.fastq" "$sra_diretoria/$sample/${sample}_R2_bbduk.fastq"

    echo "Mapping for $sample completed."
  done < "$sample_file"
}

In [86]:
map_bwa ~/Projeto_Mtb/list_isolates_SRA.txt

Mapping genomes to MTB_anc with bwa mem + sorting BAM by read name

Mapping Sample: DRR034421
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 461778 sequences (80000283 bp)...
[M::process] read 476458 sequences (80000462 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (843, 221806, 87, 561)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (117, 200, 357)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 837)
[M::mem_pestat] mean and std.dev: (242.04, 171.94)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 1077)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (178, 266, 389)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 811)
[M::mem_pestat] mean and std.dev: (294.03, 155.16)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 1022)
[M::mem_p

# **4. Post-Processing**

After mapping, mapped reads will be post-processed to ensure the reliability of the subsequent analysis (variant calling).

First, duplicates will be marked using `samtools markup` in the function `mark_duplicates`, and the BAM files will be indexed.

Then, for each of these BAM files, coverage will be calculated using `mosdepth` [4] to provide a measure of the depth of sequencing across the genome. In the function `calculate_bam_coverage`, only properly paired and mapped reads will be included in the coverage calculation. Moreover, only reads with a mapping quality of at least 20 will be included (the median will be used instead of the mean) and, for each sample, 4 files will be created: summary; golbal.dist; per-base.bed (zip and csi file).

After obtaining BAM coverage, the function `process_coverage_metrics` will extract and store (in a file) the coverage metrics, as well as remove and store the regions with no coverage.

In [87]:
# Duplicates marking 
mark_duplicates() {
  echo "Marking duplicates with samtools markdup and indexing BAM files"
  local sample_file="$1"
  local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads"
  local threads=$(nproc)

  if [ ! -f "$sample_file" ]; then
    echo "Sample '$sample_file' file not found."
    return 1
  fi

  while IFS= read -r sample || [[ -n $sample ]]; do
    if [ ! -f "$sra_diretoria/$sample/$sample.bam" ]; then
      echo "No BAM file found for: $sample."
      continue
    fi

    samtools fixmate -m -@ 2 "$sra_diretoria/$sample/$sample.bam" -u - \
    | samtools sort -u -@ 2 - \
    | samtools markdup --include-fails -S --mode s -@ 2 - -O bam,level=1 "$sra_diretoria/$sample/${sample}_markdup.bam" \
    && samtools index -@ 2 "$sra_diretoria/$sample/${sample}_markdup.bam" \
    && rm "$sra_diretoria/$sample/$sample.bam"

  done < "$sample_file"

  echo "Duplicate marking and indexing finished"
}

In [88]:
mark_duplicates ~/Projeto_Mtb/list_isolates_SRA.txt

Marking duplicates with samtools markdup and indexing BAM files
[bam_sort_core] merging from 0 files and 2 in-memory blocks...
[bam_sort_core] merging from 0 files and 2 in-memory blocks...
[bam_sort_core] merging from 0 files and 2 in-memory blocks...
[bam_sort_core] merging from 0 files and 2 in-memory blocks...
Duplicate marking and indexing finished


In [7]:
calculate_bam_coverage() {
  local sample_file="$1"
  local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads"
  local threads=$(nproc)
  local mosdepth_bin="$CONDA_PREFIX/bin/mosdepth"

  echo "Calculating bam coverage with mosdepth in parallel"
  while IFS= read -r sample || [[ -n $sample ]]; do
    if [ -z "$sample" ]; then
      continue 
    fi

    if [ ! -d "$sra_diretoria/$sample" ]; then
      echo "Directory not found for: $sample"
      continue
    fi

    bam_file="$sra_diretoria/$sample/${sample}_markdup.bam"
    if [ ! -f "$bam_file" ]; then
      echo "No BAM found for: $sample"
      continue
    fi

    pushd "$sra_diretoria/$sample" > /dev/null || continue
    echo "Calculating coverage for ${sample}_markdup.bam"
    echo "$bam_file" | parallel -j0 --colsep="\t" "${mosdepth_bin}" --flag 3844 --mapq 20 --use-median --threads "$threads" "${sample}_markdup.bam"

    echo "BAM coverage calculated for $sample"

    popd > /dev/null || continue
  done < "$sample_file"

  echo "Coverage calculation completed"
}

In [10]:
calculate_bam_coverage ~/Projeto_Mtb/list_isolates_SRA.txt

Calculating bam coverage with mosdepth in parallel
Calculating coverage for DRR034421_markdup.bam
Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

To silence the citation notice: run 'parallel --citation'.

BAM coverage calculated for DRR034421
Calculating coverage for DRR130077_markdup.bam
Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COS

In [11]:
process_coverage_metrics() {
    local sample_file="$1"
    local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads"

    while IFS= read -r sample || [[ -n $sample ]]; do
        local summary_file="$sra_diretoria/$sample/${sample}_markdup.bam.mosdepth.summary.txt"
        local dist_file="$sra_diretoria/$sample/${sample}_markdup.bam.mosdepth.global.dist.txt"
        
        if [ ! -f "$summary_file" ] || [ ! -f "$dist_file" ]; then
            echo "Coverage metric files not found for sample $sample"
            continue
        fi

        echo "Extracting coverage metrics for $sample"
        
        local median=$(awk 'NR>1 && NR%2==1 {print $4}' "$summary_file")
        local cov10=$(grep -m1 '^10\t' "$dist_file" | awk '{print $3}')
        echo -e "$sample\t$median\t$cov10" > "$sra_diretoria/$sample/${sample}_coveragecheck.tsv"

        echo "Extracting genomic positions without mapped coverage for $sample"
        
        grep -h -E '^[^[:space:]]+[[:space:]]+0$' "$sra_diretoria/$sample/"*per-base* > "$sra_diretoria/$sample/${sample}_zerocoverageregions.tsv"

        rm -f "$sra_diretoria/$sample/"*mosdepth* "$sra_diretoria/$sample/"*per-base*
    
    done < "$sample_file" 
}


In [12]:
process_coverage_metrics ~/Projeto_Mtb/list_isolates_SRA.txt

Extracting coverage metrics for DRR034421


Extracting genomic positions without mapped coverage for DRR034421
Extracting coverage metrics for DRR130077
Extracting genomic positions without mapped coverage for DRR130077
Extracting coverage metrics for DRR130078
Extracting genomic positions without mapped coverage for DRR130078
Extracting coverage metrics for DRR130093
Extracting genomic positions without mapped coverage for DRR130093


# **5. Variant Calling**

The processed BAM files will be used for the variant calling to ensure the use and call of realiable data.

For variant identification, different strategies were implemented to compare and integrate the information of different callers in the final result.   

## Simple variant calling with `bcftools mpileup`

Firstly, the most simple variant calling was performed using the `bcftools mpileup` and `bcftools call` commands with no extra options in order to obtain the most basic and simple VCF file [5]. 

In [33]:
variant_calling_simple() {
    local sample_file="$1"
    local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads"
    local ref_file="$HOME/Projeto_Mtb/NGS_helper_files/MTB_anc.fasta"
    
    while IFS= read -r sample || [[ -n $sample ]]; do
        echo "Running variant calling for sample: $sample" 
        bcftools mpileup -Ou -f "$ref_file" "$sra_diretoria/$sample/${sample}_markdup.bam" \
        | bcftools call --ploidy 1 -Ov -vc -o "$sra_diretoria/$sample/${sample}_simple.vcf"
    done < "$sample_file" 
}

In [34]:
variant_calling_simple ~/Projeto_Mtb/list_isolates_SRA.txt

Running variant calling for sample: DRR034421
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
Running variant calling for sample: DRR130077
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
Running variant calling for sample: DRR130078
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250
Running variant calling for sample: DRR130093
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250


## Variant Calling with Filters and Annotations using `bcftools mpileup, call, norm, annotate`

The second strategy used consisted in the use of `bcftools mpileup` and `call`, as well as `norm` and `annotate` [5], to apply filters (e.g. use of reads with a quality of alignment and mapping superior to 20), normalize the representation of the variants called, and add information for each one regarding, for example, lineage, excluded regions and mapability. 

In [152]:
create_intervals() {
    local threads=24
    local output_file="$HOME/Projeto_Mtb/NGS_helper_files/intervals_${threads}threads.bed"
    
    echo "Creating equally-sized intervals file for the reference genome"
    bedtools makewindows -g ~/Projeto_Mtb/NGS_helper_files/MTB_anc.fasta.fai -n "$threads" -i winnum \
    | awk '{print $1"\t"$2+1"\t"$3}' > "$output_file"
    cat -n "$output_file"
}

run_variant_calling() {
    local sample="$1"
    local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads"
    local output_dir="$sra_diretoria/$sample"
    local output_prefix="${sample}_bcftools_varsonly"
    local ref_file="$HOME/Projeto_Mtb/NGS_helper_files/MTB_anc.fasta"
    local threads=$(nproc)
    local intervals_file="$HOME/Projeto_Mtb/NGS_helper_files/intervals_24threads.bed"

    echo "Running variant calling for sample: $sample"
    bcftools mpileup -f "$ref_file" "$sra_diretoria/$sample/${sample}_markdup.bam" \
    --count-orphans \
    --no-BAQ --min-MQ 20 --min-BQ 20 \
    --regions-file "$intervals_file" \
    --annotate AD,ADF,ADR,DP,SP,SCR,INFO/AD,INFO/ADF,INFO/ADR,INFO/SCR \
    --threads "$threads" --output-type u \
    | bcftools call --ploidy 1 \
    --keep-alts --keep-masked-ref \
    --multiallelic-caller \
    --variants-only \
    --threads "$threads" --output-type u \
    | bcftools norm --fasta-ref "$ref_file" \
    --multiallelics - --keep-sum AD \
    --threads "$threads" --output-type v \
    | bcftools annotate \
    --annotations ~/Projeto_Mtb/NGS_helper_files/excludedloci_RLC2021_annot.tab.gz \
    --header-lines ~/Projeto_Mtb/NGS_helper_files/excludedloci_RLC2021_annot.header \
    --columns CHROM,FROM,TO,RLC_tag \
    --threads "$threads" --output-type u \
    | bcftools annotate \
    --annotations ~/Projeto_Mtb/NGS_helper_files/blindspots_mappability_marin2021_annot.tab.gz \
    --header-lines ~/Projeto_Mtb/NGS_helper_files/blindspots_mappability_marin2021_annot.header \
    --columns CHROM,FROM,TO,Mappability \
    --threads "$threads" --output-type u \
    | bcftools annotate \
    --annotations ~/Projeto_Mtb/NGS_helper_files/lineagesnps_annot.tab.gz \
    --header-lines ~/Projeto_Mtb/NGS_helper_files/lineagesnps_annot.header \
    --columns CHROM,POS,REF,ALT,Lineage_tag \
    --threads "$threads" --output-type u \
    | bcftools annotate \
    --annotations ~/Projeto_Mtb/NGS_helper_files/iedbepitopes_annot.tab.gz \
    --header-lines ~/Projeto_Mtb/NGS_helper_files/iedbepitopes_annot.header \
    --columns CHROM,POS,REF,ALT,IEDB_tag \
    --merge-logic IEDB_tag:unique \
    --threads "$threads" --output-type v \
    | bcftools annotate --set-id +'%CHROM:%POS' \
    | bgzip > "$output_dir/${output_prefix}_annotated.vcf.gz"

    if [ -s "$output_dir/${output_prefix}_annotated.vcf.gz" ]; then
        echo "Indexing the final VCF file"
        tabix -p vcf "$output_dir/${output_prefix}_annotated.vcf.gz"
    else
        echo "Error: Failed to create the final VCF file for sample: $sample"
    fi
}

variant_calling() {
    local sample_file="$1"
    local threads=$(nproc)
    echo "Creating equally-sized intervals file for the reference genome"
    create_intervals
    
    echo "Variant calling with bcftools parallel jobs"
    while IFS= read -r sample || [[ -n $sample ]]; do
        echo "Running variant calling for sample: $sample"
        run_variant_calling "$sample"
    done < "$sample_file"    
}

In [153]:
variant_calling ~/Projeto_Mtb/list_isolates_SRA.txt

Creating equally-sized intervals file for the reference genome
Creating equally-sized intervals file for the reference genome
     1	MTB_anc	1	183813
     2	MTB_anc	183814	367626
     3	MTB_anc	367627	551439
     4	MTB_anc	551440	735252
     5	MTB_anc	735253	919065
     6	MTB_anc	919066	1102878
     7	MTB_anc	1102879	1286691
     8	MTB_anc	1286692	1470504
     9	MTB_anc	1470505	1654317
    10	MTB_anc	1654318	1838130
    11	MTB_anc	1838131	2021943
    12	MTB_anc	2021944	2205756
    13	MTB_anc	2205757	2389569
    14	MTB_anc	2389570	2573382
    15	MTB_anc	2573383	2757195
    16	MTB_anc	2757196	2941008
    17	MTB_anc	2941009	3124821
    18	MTB_anc	3124822	3308634
    19	MTB_anc	3308635	3492447
    20	MTB_anc	3492448	3676260
    21	MTB_anc	3676261	3860073
    22	MTB_anc	3860074	4043886
    23	MTB_anc	4043887	4227699
    24	MTB_anc	4227700	4411532
Variant calling with bcftools parallel jobs
Running variant calling for sample: DRR034421
Running variant calling for sample: DRR034421
[mpileup] 

## LoFreq + bcftools

Due to its high sensitivity in detecting low-frequency variants, crucial for identifying minor variants, the LoFreq tool was also employed [6]. The process involved two main steps: post-processing and variant calling. In the post-processing step, the `lofreq_postprocessing` function performed indel realignment on duplicated BAM files to refine read alignments around indels, followed by indexing the realigned BAM files. Subsequently, the `LoFreq_calling` function used `lofreq call-parallel` to detect variants in the realigned BAM files, producing VCF files with annotated and high-confidence variant calls. 

Additionally, `bcftools` was employed to further refine the variant calls by applying stringent filters and incorporating comprehensive annotations to the VCF files generated. This combined approach generated VCF files with low-frequency variants annotated with information regarding excluded loci, mappability, lineage-specific SNPs, and epitope region

In [11]:
lofreq_postprocessing() { 
    local sample_file="$1" 
    local genome_reference="$HOME/Projeto_Mtb/NGS_helper_files/MTB_anc.fasta"
    local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads" 
    while IFS= read -r sample || [[ -n $sample ]]; do 
        if [ ! -f "$sra_diretoria/$sample/${sample}_markdup.bam" ]; then 
            echo "No markdup.bam found for: $sample." 
            continue 
        fi 
        lofreq indelqual --dindel -f $genome_reference -o "$sra_diretoria/$sample/${sample}_realigned.bam" "$sra_diretoria/$sample/${sample}_markdup.bam"
        samtools index "$sra_diretoria/$sample/${sample}_realigned.bam"
    done < "$sample_file" 
    echo 'Post-processing with LoFreq completed.'
}

LoFreq_calling() {
    local sample_file="$1" 
    local genome_reference="$HOME/Projeto_Mtb/NGS_helper_files/MTB_anc.fasta"
    local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads" 
    while IFS= read -r sample || [[ -n $sample ]]; do 
        if [ ! -f "$sra_diretoria/$sample/${sample}_realigned.bam" ]; then 
            echo "No realigned.bam found for: $sample." 
            continue 
        fi 
        echo "Performing variant calling for $sample"
        lofreq call-parallel --pp-threads 4 -f $genome_reference -o "$sra_diretoria/$sample/${sample}_lofreq.vcf" "$sra_diretoria/$sample/${sample}_realigned.bam" 
    done < "$sample_file"  
    echo 'Variant calling using LoFreq completed.'
}

In [13]:
lofreq_postprocessing ~/Projeto_Mtb/list_isolates_SRA.txt

Post-processing with LoFreq completed.


In [14]:
LoFreq_calling ~/Projeto_Mtb/list_isolates_SRA.txt

Performing variant calling for DRR034421
INFO [2024-06-16 22:44:16,019]: Using 4 threads with following basic args: lofreq call -f /home/ritanobrega00/Projeto_Mtb/NGS_helper_files/MTB_anc.fasta /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_realigned.bam

INFO [2024-06-16 22:44:16,028]: Adding 12 commands to mp-pool
Number of substitution tests performed: 619218
Number of indel tests performed: 0
INFO [2024-06-16 22:49:51,060]: Executing lofreq filter -i /tmp/lofreq2_call_parallel3c_gxp68/concat.vcf.gz -o /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_lofreq.vcf --snvqual-thresh 78 --indelqual-thresh 20

Performing variant calling for DRR130077
INFO [2024-06-16 22:49:51,174]: Using 4 threads with following basic args: lofreq call -f /home/ritanobrega00/Projeto_Mtb/NGS_helper_files/MTB_anc.fasta /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130077/DRR130077_realigned.bam

INFO [2024-06-16 22:49:51,184]: Adding 12 commands to mp-pool
Number of substi

## LoFreq + Bcftools

In [2]:
lofreq_bcftools() {
    local sample_file="$1"
    local genome_reference="$HOME/Projeto_Mtb/NGS_helper_files/MTB_anc.fasta"
    local sra_directory="$HOME/Projeto_Mtb/Sample_reads"

    echo "Variant calling using LoFreq and bcftools"
    while IFS= read -r sample || [[ -n $sample ]]; do
        local sample_dir="$sra_directory/$sample"
        local lofreq_vcf="$sample_dir/${sample}_lofreq.vcf"
        local bcftools_output="$sample_dir/${sample}_bcftools_lofreq.vcf.gz"

        if [ -s "$lofreq_vcf" ]; then
            echo "Running bcftools processing for sample: $sample"
            bgzip -c "$sample_dir/${sample}_lofreq.vcf" > "$sample_dir/${sample}_lofreq.vcf.gz"
            tabix -p vcf "$sample_dir/${sample}_lofreq.vcf.gz"
            bcftools annotate --annotations ~/Projeto_Mtb/NGS_helper_files/excludedloci_RLC2021_annot.tab.gz \
                             --header-lines ~/Projeto_Mtb/NGS_helper_files/excludedloci_RLC2021_annot.header \
                             --columns CHROM,FROM,TO,RLC_tag \
                             --threads "$(nproc)" --output-type u "$lofreq_vcf.gz" \
            | bcftools annotate --annotations ~/Projeto_Mtb/NGS_helper_files/blindspots_mappability_marin2021_annot.tab.gz \
                                --header-lines ~/Projeto_Mtb/NGS_helper_files/blindspots_mappability_marin2021_annot.header \
                                --columns CHROM,FROM,TO,Mappability \
                                --threads "$(nproc)" --output-type u \
            | bcftools annotate --annotations ~/Projeto_Mtb/NGS_helper_files/lineagesnps_annot.tab.gz \
                                --header-lines ~/Projeto_Mtb/NGS_helper_files/lineagesnps_annot.header \
                                --columns CHROM,POS,REF,ALT,Lineage_tag \
                                --threads "$(nproc)" --output-type u \
            | bcftools annotate --annotations ~/Projeto_Mtb/NGS_helper_files/iedbepitopes_annot.tab.gz \
                                --header-lines ~/Projeto_Mtb/NGS_helper_files/iedbepitopes_annot.header \
                                --columns CHROM,POS,REF,ALT,IEDB_tag \
                                --merge-logic IEDB_tag:unique \
                                --threads "$(nproc)" --output-type v \
            | bcftools annotate --set-id +'%CHROM:%POS' \
            | bgzip > "$bcftools_output"

            if [ -s "$bcftools_output" ]; then
                echo "Indexing the final annotated VCF file for sample: $sample"
                tabix -p vcf "$bcftools_output"
            else
                echo "Error: Failed to create the final annotated VCF file for sample: $sample"
            fi
        else
            echo "Error: Failed to generate LoFreq VCF for sample: $sample"
        fi
    done < "$sample_file"

    echo "Variant calling using LoFreq and bcftools completed."
}

In [3]:
lofreq_bcftools ~/Projeto_Mtb/list_isolates_SRA.txt

Variant calling using LoFreq and bcftools
Running bcftools processing for sample: DRR034421
Indexing the final annotated VCF file for sample: DRR034421
Running bcftools processing for sample: DRR130077
Indexing the final annotated VCF file for sample: DRR130077
Running bcftools processing for sample: DRR130078
Indexing the final annotated VCF file for sample: DRR130078
Running bcftools processing for sample: DRR130093
Indexing the final annotated VCF file for sample: DRR130093
Variant calling using LoFreq and bcftools completed.


# Compare VCF files from different variant callers

In [12]:
tabix -p vcf "$HOME/Projeto_Mtb/Sample_reads/DRR130078/DRR130078_bcftools_lofreq.vcf.gz"

[tabix] the index file exists. Please use '-f' to overwrite.


: 1

In [16]:
vcf_files=("DRR130078_simple.vcf" "DRR130078_bcftools_varsonly_annotated.vcf" "DRR130078_lofreq.vcf")
for vcf_file in "${vcf_files[@]}"
do
    echo "Variants identified in $vcf_file:"
    grep -c "^MTB_anc" $HOME/Projeto_Mtb/Sample_reads/DRR130078/"$vcf_file"
done

Variants identified in DRR130078_simple.vcf:
1576
Variants identified in DRR130078_bcftools_varsonly_annotated.vcf:
1640
Variants identified in DRR130078_lofreq.vcf:
1743


In [45]:
vcf_files=("DRR130077_simple.vcf" "DRR130077_bcftools_varsonly_annotated.vcf" "DRR130077_lofreq.vcf")
for vcf_file in "${vcf_files[@]}"
do
    echo "Variants identified in $vcf_file:"
    grep -c "^MTB_anc" $HOME/Projeto_Mtb/Sample_reads/DRR130077/"$vcf_file"
done

Variants identified in: DRR130077_simple.vcf
1532
Variants identified in: DRR130077_bcftools_varsonly_annotated.vcf
1560
Variants identified in: DRR130077_lofreq.vcf
1616


In [42]:
vcf_files=("DRR034421_simple.vcf" "DRR034421_bcftools_varsonly_annotated.vcf" "DRR034421_lofreq.vcf")
for vcf_file in "${vcf_files[@]}"
do
    echo "Variants identified in $vcf_file:"
    grep -c "^MTB_anc" $HOME/Projeto_Mtb/Sample_reads/DRR034421/"$vcf_file"
done


Variants identified in: DRR034421_simple.vcf
1417
Variants identified in: DRR034421_bcftools_varsonly_annotated.vcf
1472
Variants identified in: DRR034421_lofreq.vcf
1346


In [46]:
vcf_files=("DRR130093_simple.vcf" "DRR130093_bcftools_varsonly_annotated.vcf" "DRR130093_lofreq.vcf.gz")
for vcf_file in "${vcf_files[@]}"
do
    echo "Variants identified in $vcf_file:"
    
    grep -c "^MTB_anc" $HOME/Projeto_Mtb/Sample_reads/DRR130093/"$vcf_file"
done

Variants identified in DRR130093_simple.vcf:
1571
Variants identified in DRR130093_bcftools_varsonly_annotated.vcf:
1645
Variants identified in DRR130093_lofreq.vcf:
1723


In [6]:
comparing_VCF(){
    local sample="$1" 
    local sra_diretoria="$HOME/Projeto_Mtb/Sample_reads" 

    echo "Comparing VCF files for $sample:"
    bgzip -c "$sra_diretoria/${sample}/${sample}_simple.vcf" > "$sra_diretoria/${sample}/${sample}_simple.vcf.gz"
    bgzip -c "$sra_diretoria/${sample}/${sample}_bcftools_varsonly_annotated.vcf" > "$sra_diretoria/${sample}/${sample}_bcftools_varsonly_annotated.vcf.gz"

    tabix -p vcf "$sra_diretoria/${sample}/${sample}_simple.vcf.gz"
    tabix -p vcf "$sra_diretoria/${sample}/${sample}_bcftools_varsonly_annotated.vcf.gz"
    tabix -p vcf "$sra_diretoria/${sample}/${sample}_bcftools_lofreq.vcf.gz"
    
    vcf-compare "$sra_diretoria/${sample}/${sample}_simple.vcf.gz" "$sra_diretoria/${sample}/${sample}_bcftools_varsonly_annotated.vcf.gz" "$sra_diretoria/${sample}/${sample}_bcftools_lofreq.vcf.gz" 
}

In [7]:
comparing_VCF DRR130078

Comparing VCF files for DRR130078:
[tabix] the index file exists. Please use '-f' to overwrite.
# This file was generated by vcf-compare.
# The command line was: vcf-compare(v0.1.14-12-gcdb80b8) /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130078/DRR130078_simple.vcf.gz /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130078/DRR130078_bcftools_varsonly_annotated.vcf.gz /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130078/DRR130078_bcftools_lofreq.vcf.gz
#
#VN 'Venn-Diagram Numbers'. Use `grep ^VN | cut -f 2-` to extract this part.
#VN The columns are: 
#VN        1  .. number of sites unique to this particular combination of files
#VN        2- .. combination of files and space-separated number, a fraction of sites in the file
VN	5	/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130078/DRR130078_bcftools_lofreq.vcf.gz (0.3%)	/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130078/DRR130078_bcftools_varsonly_annotated.vcf.gz (0.3%)
VN	19	/home/ritanobrega00/Projeto_Mtb/Sample_reads/D

In [60]:
comparing_VCF DRR130077

Comparing VCF files for DRR130077:
# This file was generated by vcf-compare.
# The command line was: vcf-compare(v0.1.14-12-gcdb80b8) /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130077/DRR130077_simple.vcf.gz /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130077/DRR130077_bcftools_varsonly_annotated.vcf.gz /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130077/DRR130077_lofreq.vcf.gz
#
#VN 'Venn-Diagram Numbers'. Use `grep ^VN | cut -f 2-` to extract this part.
#VN The columns are: 
#VN        1  .. number of sites unique to this particular combination of files
#VN        2- .. combination of files and space-separated number, a fraction of sites in the file
VN	3	/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130077/DRR130077_bcftools_varsonly_annotated.vcf.gz (0.2%)	/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130077/DRR130077_lofreq.vcf.gz (0.2%)
VN	21	/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130077/DRR130077_simple.vcf.gz (1.4%)
VN	24	/home/ritanobrega00/Projeto_Mtb/S

In [8]:
comparing_VCF DRR034421

Comparing VCF files for DRR034421:
[tabix] the index file exists. Please use '-f' to overwrite.
# This file was generated by vcf-compare.
# The command line was: vcf-compare(v0.1.14-12-gcdb80b8) /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_simple.vcf.gz /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_bcftools_varsonly_annotated.vcf.gz /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_bcftools_lofreq.vcf.gz
#
#VN 'Venn-Diagram Numbers'. Use `grep ^VN | cut -f 2-` to extract this part.
#VN The columns are: 
#VN        1  .. number of sites unique to this particular combination of files
#VN        2- .. combination of files and space-separated number, a fraction of sites in the file
VN	4	/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_bcftools_lofreq.vcf.gz (0.3%)	/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR034421/DRR034421_bcftools_varsonly_annotated.vcf.gz (0.3%)
VN	4	/home/ritanobrega00/Projeto_Mtb/Sample_reads/DR

In [9]:
comparing_VCF DRR130093

Comparing VCF files for DRR130093:
[tabix] the index file exists. Please use '-f' to overwrite.
# This file was generated by vcf-compare.
# The command line was: vcf-compare(v0.1.14-12-gcdb80b8) /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130093/DRR130093_simple.vcf.gz /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130093/DRR130093_bcftools_varsonly_annotated.vcf.gz /home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130093/DRR130093_bcftools_lofreq.vcf.gz
#
#VN 'Venn-Diagram Numbers'. Use `grep ^VN | cut -f 2-` to extract this part.
#VN The columns are: 
#VN        1  .. number of sites unique to this particular combination of files
#VN        2- .. combination of files and space-separated number, a fraction of sites in the file
VN	3	/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130093/DRR130093_bcftools_lofreq.vcf.gz (0.2%)	/home/ritanobrega00/Projeto_Mtb/Sample_reads/DRR130093/DRR130093_bcftools_varsonly_annotated.vcf.gz (0.2%)
VN	20	/home/ritanobrega00/Projeto_Mtb/Sample_reads/D

# References:

[1] https://github.com/ncbi/sra-tools/wiki/08.-prefetch-and-fasterq-dump

[2] https://jgi.doe.gov/data-and-tools/software-tools/bbtools/bb-tools-user-guide/bbduk-guide/

[3]  https://github.com/lh3/bwa

[4] https://github.com/brentp/mosdepth

[5] https://samtools.github.io/bcftools/bcftools.html

[6] Wilm, A., Aw, P. P., Bertrand, D., Yeo, G. H., Ong, S. H., Wong, C. H., Khor, C. C., Petric, R., Hibberd, M. L., & Nagarajan, N. (2012). LoFreq: a sequence-quality aware, ultra-sensitive variant caller for uncovering cell-population heterogeneity from high-throughput sequencing datasets. Nucleic acids research, 40(22), 11189–11201. https://doi.org/10.1093/nar/gks918
